In [1]:
%run ../yp_utils.py

# Initial setup

In [2]:
paper_pmid = 29351850
paper_name = 'jaeger_ideker_2018' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [4]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data

In [30]:
sheet_names = ['GLU GAL Fitness','GLU GAL Fluorescence Data','CPY Fluorescent Data']
data_cols = [['SC-GLU','SC-GAL'],['GLU','GAL','GLU.1','GAL.1'],['Ctrl (fluo.)','CPY-GFP (fluo.)']]

In [31]:
original_data_list = []
for si, s in enumerate(sheet_names):
    original_data = pd.read_excel('raw_data/Calico_Data_v1.xlsx', sheet_name=s, skiprows=1)
    print('Original data dimensions: %d x %d' % (original_data.shape))
    original_data['orf'] = original_data['ORF'].astype(str)
    original_data['orf'] = clean_orf(original_data['orf'])
    original_data.loc[original_data['orf']=='YLR287-A','orf'] = 'YLR287C-A'
    original_data['orf'] = translate_sc(original_data['orf'], to='orf')
    t = looks_like_orf(original_data['orf'])
    print(original_data.loc[~t,])
    original_data.set_index('orf', inplace=True)
    
    original_data = original_data.loc[:,data_cols[si]]
    original_data = original_data.groupby(original_data.index).mean()
    
    original_data_list.append(original_data)

Original data dimensions: 5835 x 4
Empty DataFrame
Columns: [Gene, ORF, SC-GLU, SC-GAL, orf]
Index: []
Original data dimensions: 5807 x 6
Empty DataFrame
Columns: [Gene, ORF, GLU, GAL, GLU.1, GAL.1, orf]
Index: []
Original data dimensions: 5807 x 8
Empty DataFrame
Columns: [Gene, ORF, Ctrl (size), CPY-GFP (size), Ctrl (fluo.), CPY-GFP (fluo.), CPY-NES-GFP (fluo.), CPY-GFP san1D (fluo.), orf]
Index: []


In [32]:
original_data1, original_data2, original_data3 = original_data_list

In [33]:
original_data = original_data1.join(original_data2, how='outer', lsuffix='_1', rsuffix='_2')
original_data = original_data.join(original_data3, how='outer', lsuffix='', rsuffix='_3')

In [35]:
original_data.shape

(5830, 8)

In [34]:
original_data.head()

,SC-GLU,SC-GAL,GLU,GAL,GLU.1,GAL.1,Ctrl (fluo.),CPY-GFP (fluo.)
orf,,,,,,,,
YAL002W,-0.406263,0.245177,-0.634156,-0.319514,-0.060031,-0.122411,-1.049142,-0.415537
YAL003W,-0.196194,0.171792,-0.842585,-0.163610,-0.025241,-0.093237,-0.865258,0.572071
YAL004W,0.353014,0.293261,-0.607665,0.213934,0.098942,-0.028650,0.427745,1.372855
YAL005C,0.295915,-0.469302,-0.298648,0.196771,0.163048,-0.054926,0.669254,0.689108
YAL007C,-0.029866,0.254167,0.302599,0.154397,-0.111000,-0.011036,-0.059514,0.188327


In [36]:
original_data['Fluo'] = original_data['CPY-GFP (fluo.)'] - original_data['Ctrl (fluo.)']
original_data.drop(columns=['Ctrl (fluo.)','CPY-GFP (fluo.)'], inplace=True)

In [37]:
dataset_ids = [16153, 16154] + [16165, 16166, 16319, 16318] + [16155]

In [40]:
essentials = ~is_essential(original_data.index.values)

Loading SGD features from 2017-04-03.


In [43]:
original_data = original_data.loc[essentials.values,:]

In [44]:
original_data.shape

(4952, 7)

# Prepare the final dataset

In [45]:
data = original_data.copy()

In [46]:
datasets = datasets.reindex(index=dataset_ids)

In [47]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [48]:
data.head()

dataset_id,16153,16154,16165,16166,16319,16318,16155
data_type,value,value,value,value,value,value,value
orf,,,,,,,
YAL002W,-0.406263,0.245177,-0.634156,-0.319514,-0.060031,-0.122411,0.633605
YAL004W,0.353014,0.293261,-0.607665,0.213934,0.098942,-0.028650,0.945110
YAL005C,0.295915,-0.469302,-0.298648,0.196771,0.163048,-0.054926,0.019854
YAL007C,-0.029866,0.254167,0.302599,0.154397,-0.111000,-0.011036,0.247841
YAL008W,-0.501140,0.161716,0.713786,0.122458,0.069959,-0.006048,-0.089864


## Subset to the genes currently in SGD

In [49]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 25


In [50]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

In [51]:
data.head()

,dataset_id,16153,16154,16165,16166,16319,16318,16155
,data_type,value,value,value,value,value,value,value
gene_id,orf,,,,,,,
2,YAL002W,-0.406263,0.245177,-0.634156,-0.319514,-0.060031,-0.122411,0.633605
1863,YAL004W,0.353014,0.293261,-0.607665,0.213934,0.098942,-0.028650,0.945110
4,YAL005C,0.295915,-0.469302,-0.298648,0.196771,0.163048,-0.054926,0.019854
5,YAL007C,-0.029866,0.254167,0.302599,0.154397,-0.111000,-0.011036,0.247841
6,YAL008W,-0.501140,0.161716,0.713786,0.122458,0.069959,-0.006048,-0.089864


# Normalize

In [52]:
data_norm = normalize_phenotypic_scores(data, has_tested=True)

In [53]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [54]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

In [55]:
data_all.head()


,dataset_id,16153,16154,16165,16166,16319,16318,16155,16153,16154,16165,16166,16319,16318,16155
,data_type,value,value,value,value,value,value,value,valuez,valuez,valuez,valuez,valuez,valuez,valuez
gene_id,orf,,,,,,,,,,,,,,
2,YAL002W,-0.406263,0.245177,-0.634156,-0.319514,-0.060031,-0.122411,0.633605,-0.748932,-0.064546,-0.902801,-1.238670,-0.533078,-0.769564,0.835368
1863,YAL004W,0.353014,0.293261,-0.607665,0.213934,0.098942,-0.028650,0.945110,0.086021,-0.020049,-0.875303,0.309118,0.039638,-0.090073,1.131224
4,YAL005C,0.295915,-0.469302,-0.298648,0.196771,0.163048,-0.054926,0.019854,0.023232,-0.725717,-0.554542,0.259318,0.270585,-0.280495,0.252453
5,YAL007C,-0.029866,0.254167,0.302599,0.154397,-0.111000,-0.011036,0.247841,-0.335019,-0.056226,0.069557,0.136373,-0.716697,0.037578,0.468986
6,YAL008W,-0.501140,0.161716,0.713786,0.122458,0.069959,-0.006048,-0.089864,-0.853265,-0.141779,0.496372,0.043703,-0.064776,0.073728,0.148248


# Print out

In [56]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [57]:
# from IO.save_data_to_db3 import *

In [1]:
# save_data_to_db(data_all, paper_pmid)